In [ ]:
#in class
from queue import PriorityQueue

graph={
    'A':[('B',5),('C',8)],
    'B':[('D',10)],
    'C':[('E',3)],
    'D':[('F',7)],
    'E': [('F',2)],
    'F':[(),]
}

def bfs(graph,start,goal):
  visited = set()
  pq = PriorityQueue()
  pq.put((0,start))

  while not pq.empty():
    print('Pq : ',pq.queue)
    print('visited : ',visited)
    cost, node = pq.get()

    if node not in visited:
      print(node,end=' ')
      visited.add(node)


    if node == goal:
      print('\nGoal Found')
      return

    for neighbor,weight in graph[node]:
      if neighbor not in visited:
        pq.put((weight,neighbor))

  print('Goal Not Found')
  return False

print('bfs path')
bfs(graph,'A','F')

bfs path
Pq :  [(0, 'A')]
visited :  set()
A Pq :  [(5, 'B'), (8, 'C')]
visited :  {'A'}
B Pq :  [(8, 'C'), (10, 'D')]
visited :  {'B', 'A'}
C Pq :  [(3, 'E'), (10, 'D')]
visited :  {'C', 'B', 'A'}
E Pq :  [(2, 'F'), (10, 'D')]
visited :  {'E', 'C', 'B', 'A'}
F 
Goal Found


In [10]:
#Q1.
import heapq
from itertools import permutations

class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.rows = len(maze)
        self.cols = len(maze[0])
        self.start = None
        self.goals = []
        self._find_points()

    def _find_points(self):
        for r in range(self.rows):
            for c in range(self.cols):
                if self.maze[r][c] == 'S':
                    self.start = (r, c)
                elif self.maze[r][c] == 'G':
                    self.goals.append((r, c))

    def heuristic(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def bfs(self, start, goal):
        queue = [(0, start)]
        came_from = {start: None}
        cost_so_far = {start: 0}

        while queue:
            _, current = heapq.heappop(queue)

            if current == goal:
                break

            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next_cell = (current[0] + dr, current[1] + dc)

                if (0 <= next_cell[0] < self.rows and 0 <= next_cell[1] < self.cols and
                        self.maze[next_cell[0]][next_cell[1]] != '#' and next_cell not in cost_so_far):
                    new_cost = cost_so_far[current] + 1
                    cost_so_far[next_cell] = new_cost
                    priority = new_cost + self.heuristic(next_cell, goal)
                    heapq.heappush(queue, (priority, next_cell))
                    came_from[next_cell] = current

        return self.reconstruct_path(came_from, start, goal)

    def reconstruct_path(self, came_from, start, goal):
        path = []
        current = goal
        while current != start:
            path.append(current)
            current = came_from[current]
            if current is None:
                return []
        path.append(start)
        path.reverse()
        return path

    def find_best_path(self):
        best_order = None
        best_path = []
        best_cost = float('inf')

        for order in permutations(self.goals):
            total_path = []
            total_cost = 0
            current_start = self.start

            for goal in order:
                path = self.bfs(current_start, goal)
                total_path.extend(path[1:])
                total_cost += len(path) - 1
                current_start = goal

            if total_cost < best_cost:
                best_cost = total_cost
                best_path = total_path
                best_order = order

        return best_order, best_path, best_cost

maze = [
    ['S', '.', '.', '#', '.', '.', '.'],
    ['.', '#', '.', '#', '.', '#', '.'],
    ['.', '#', '.', '.', '.', '#', '.'],
    ['.', '.', '#', '#', '.', '.', '.'],
    ['#', '.', '#', 'G', '#', '#', '.'],
    ['.', '.', '.', '.', '.', '.', 'G'],
]

solver = MazeSolver(maze)
order, path, cost = solver.find_best_path()

if path:
    print("Path found:")
    print("Optimal Order of Goals:", order)
    print("Shortest Path:", path)
    print("Total Path Cost:", cost)
else:
    print("No path found")

Path found:
Optimal Order of Goals: ((4, 3), (5, 6))
Shortest Path: [(1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6)]
Total Path Cost: 13


In [2]:
from queue import PriorityQueue
from itertools import permutations

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost
        self.h = 0  # Heuristic
        self.f = 0  # Total

    def __lt__(self, other):
        return self.f < other.f

def heuristic(current_pos, end_pos):
    return abs(current_pos[0] - end_pos[0]) + abs(current_pos[1] - end_pos[1])

def best_first_search(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    end_node = Node(end)
    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position

        if current_pos == end:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        visited.add(current_pos)

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)

            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited:
                new_node = Node(new_pos, current_node)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h  # Best-First Search: f(n) = h(n)
                frontier.put(new_node)
                visited.add(new_pos)

    return None  # No path found

def find_shortest_path_through_goals(maze, start, goals):
    min_path = None
    min_distance = float('inf')

    for perm in permutations(goals):
        total_path = []
        total_distance = 0
        current_pos = start

        for goal in perm:
            path = best_first_search(maze, current_pos, goal)
            if path is None:
                break  # goal unreachable - skipping permutation
            total_distance += len(path) - 1
            total_path.extend(path[:-1])
            current_pos = goal

        if total_distance < min_distance:
            min_distance = total_distance
            min_path = total_path + [goals[-1]]  # last goal appended

    return min_path

maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]
start = (0, 0)
goals = [(4, 4), (2, 1), (0, 3)]

path = find_shortest_path_through_goals(maze, start, goals)
if path:
    print("Path found:", path)
else:
    print("No path found")

Path found: [(0, 0), (1, 0), (2, 0), (2, 1), (1, 1), (1, 2), (1, 3), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (0, 3)]


In [ ]:
#Q2.
import random
import time
from queue import PriorityQueue

graph = {
    'A': {'B': 4, 'C': 3},
    'B': {'E': 12, 'F': 5},
    'C': {'D': 7, 'E': 10},
    'D': {'E': 2},
    'E': {'G': 5},
    'F': {'G': 16},
    'G': {},
}

heuristic = {'A': 14, 'B': 12, 'C': 11, 'D': 6, 'E': 4, 'F': 11, 'G': 0}

def update_edge_costs(graph):
    for node in graph:
        for neighbor in graph[node]:
            graph[node][neighbor] = random.randint(1, 15)

def a_star_dynamic(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put((heuristic[start], start))
    visited = set()
    g_costs = {start: 0}
    came_from = {start: None}

    while not frontier.empty():
        _, current_node = frontier.get()

        if current_node in visited:
            continue

        print(f"Visiting: {current_node}")
        visited.add(current_node)

        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = came_from[current_node]
            path.reverse()
            print(f"\nGoal found! Optimal Path: {path}")
            return

        update_edge_costs(graph)
        print("Updated Edge Costs:", graph)

        for neighbor, cost in graph[current_node].items():
            new_g_cost = g_costs[current_node] + cost
            f_cost = new_g_cost + heuristic[neighbor]

            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                came_from[neighbor] = current_node
                frontier.put((f_cost, neighbor))

        time.sleep(1)

    print("Goal not found")

print("\nRunning Dynamic A* Search:")
a_star_dynamic(graph, 'A', 'G')


Running Dynamic A* Search:
Visiting: A
Updated Edge Costs: {'A': {'B': 12, 'C': 11}, 'B': {'E': 13, 'F': 13}, 'C': {'D': 6, 'E': 1}, 'D': {'E': 13}, 'E': {'G': 8}, 'F': {'G': 6}, 'G': {}}
Visiting: C
Updated Edge Costs: {'A': {'B': 9, 'C': 2}, 'B': {'E': 12, 'F': 9}, 'C': {'D': 3, 'E': 5}, 'D': {'E': 10}, 'E': {'G': 11}, 'F': {'G': 15}, 'G': {}}
Visiting: D
Updated Edge Costs: {'A': {'B': 3, 'C': 7}, 'B': {'E': 1, 'F': 12}, 'C': {'D': 1, 'E': 5}, 'D': {'E': 5}, 'E': {'G': 7}, 'F': {'G': 8}, 'G': {}}
Visiting: E
Updated Edge Costs: {'A': {'B': 3, 'C': 6}, 'B': {'E': 1, 'F': 4}, 'C': {'D': 6, 'E': 6}, 'D': {'E': 14}, 'E': {'G': 13}, 'F': {'G': 15}, 'G': {}}
Visiting: B
Updated Edge Costs: {'A': {'B': 2, 'C': 3}, 'B': {'E': 8, 'F': 1}, 'C': {'D': 13, 'E': 1}, 'D': {'E': 14}, 'E': {'G': 1}, 'F': {'G': 5}, 'G': {}}
Visiting: F
Updated Edge Costs: {'A': {'B': 5, 'C': 3}, 'B': {'E': 1, 'F': 1}, 'C': {'D': 7, 'E': 13}, 'D': {'E': 13}, 'E': {'G': 3}, 'F': {'G': 14}, 'G': {}}
Visiting: G

Goal 

In [6]:
#Q3.
import heapq

class DeliveryRouteOptimizer:
    def __init__(self, locations, time_windows, start):
        self.locations = locations
        self.time_windows = time_windows
        self.start = start

    def distance(self, loc1, loc2):
        x1, y1 = self.locations[loc1]
        x2, y2 = self.locations[loc2]
        return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

    def heuristic(self, loc, current_time):
        tw_start, tw_end = self.time_windows[loc]
        urgency = max(0, tw_end - current_time)
        return self.distance(self.start, loc) + urgency * 2

    def optimize_route(self):
        queue = []
        visited = set()
        route = []
        current_location = self.start
        current_time = 0

        heapq.heappush(queue, (0, current_location, current_time))

        while queue:
            _, current_location, current_time = heapq.heappop(queue)

            if current_location in visited:
                continue

            visited.add(current_location)
            route.append((current_location, current_time))

            next_locations = [loc for loc in self.locations if loc not in visited]
            for loc in next_locations:
                travel_time = self.distance(current_location, loc)
                arrival_time = current_time + travel_time

                if self.time_windows[loc][0] <= arrival_time <= self.time_windows[loc][1]:
                    priority = self.heuristic(loc, arrival_time)
                    heapq.heappush(queue, (priority, loc, arrival_time))

        return route

locations = {'A': (0, 0), 'B': (2, 3), 'C': (5, 4), 'D': (6, 1)}
time_windows = {'A': (0, 10), 'B': (3, 8), 'C': (5, 12), 'D': (7, 15)}
optimizer = DeliveryRouteOptimizer(locations, time_windows, 'A')
route = optimizer.optimize_route()
print("Optimized Delivery Route:", route)

Optimized Delivery Route: [('A', 0), ('B', 3.605551275463989), ('C', 6.767828935632369), ('D', 9.930106595800748)]


In [20]:
#Q4.
import heapq
import random

class CityMap:
    def __init__(self, graph, traffic_data):
        self.graph = graph
        self.traffic_data = traffic_data

    def heuristic(self, node, goal):
        x1, y1 = city_coordinates[node]
        x2, y2 = city_coordinates[goal]
        return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

    def get_travel_time(self, u, v):
        return self.traffic_data.get((u, v), self.traffic_data.get((v, u), None))

    def a_star_search(self, start, goal):
        priority_queue = []
        heapq.heappush(priority_queue, (0, start))
        came_from = {start: None}
        cost_so_far = {start: 0}

        while priority_queue:
            _, current = heapq.heappop(priority_queue)

            if current == goal:
                break

            for neighbor, base_travel_time in self.graph.get(current, []):
                travel_time = self.get_travel_time(current, neighbor)
                if not travel_time:
                    travel_time = base_travel_time
                new_cost = cost_so_far[current] + travel_time

                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    priority = new_cost + self.heuristic(neighbor, goal)
                    heapq.heappush(priority_queue, (priority, neighbor))
                    came_from[neighbor] = current

        return self.reconstruct_path(came_from, start, goal)

    def reconstruct_path(self, came_from, start, goal):
        path = []
        current = goal
        while current != start:
            path.append(current)
            current = came_from.get(current)
            if current is None:
                return []
        path.append(start)
        path.reverse()
        return path

    def update_traffic(self):
        for edge in self.traffic_data:
            self.traffic_data[edge] = random.randint(1, 10)

city_coordinates = {
    'A': (0, 0), 'B': (2, 4), 'C': (3, 1), 'D': (5, 5), 'E': (7, 3)
}

city_graph = {
    'A': [('B', 4), ('C', 2)],
    'B': [('A', 4), ('D', 5), ('E', 10)],
    'C': [('A', 2), ('D', 8)],
    'D': [('B', 5), ('C', 8), ('E', 2)],
    'E': [('B', 10), ('D', 2)]
}

traffic_data = {
    ('A', 'B'): 4, ('A', 'C'): 2, ('B', 'D'): 5, ('B', 'E'): 10,
    ('C', 'D'): 8, ('D', 'E'): 2
}

city = CityMap(city_graph, traffic_data)

start, goal = 'A', 'E'
print("Initial Path:", city.a_star_search(start, goal))

city.update_traffic()
print("Updated Traffic Data:", city.traffic_data)
print("New Path After Traffic Update:", city.a_star_search(start, goal))

Initial Path: ['A', 'B', 'D', 'E']
Updated Traffic Data: {('A', 'B'): 2, ('A', 'C'): 8, ('B', 'D'): 9, ('B', 'E'): 2, ('C', 'D'): 8, ('D', 'E'): 4}
New Path After Traffic Update: ['A', 'B', 'E']
